# Mapping Population to Pixels


*MIT Megacity Logistics Lab*(c): Daniel Merchan <dmerchan@mit.edu>, Andre Snoeck <asnoeck@mit.edu>



**Summary**: This script can be used to map LandScan population cells to a list of pixels

Input data requirements:
- Origin point coordinates (lower left corner of the pixel grid)
- Number of kilometers in the horizontal axis (longitude)
- Number of kilometers in the veritcal axis (latitute)
- Populations files

**Case Studies**: Sao Paulo, San Francisco, Denver

This script processes and analyses two sets of data:
1. GeoSpatial data - Maps population data (LandScan) to create a rectangular grid over a city or ***pixels***. 

The total size of the grid must be defined by the user based upon each case study. For instance, for **Sao Paulo**, the size to cover the entire metropolitan area is approximately 75 km (vertical axis) x 100 km (horizontal axis). 

Furthremore, the size/area of each pixel might vary based on visualization/modeling considerations. The minimum pixel size for this code is 1 sq.km. 

** Run time **: ~ 2-30  minutes (depending on the city size)


## References

- LandsScan:
This product was made utilizing the LandScan (insert dataset year)™ High Resolution global Population Data Set copyrighted by UT-Battelle, LLC, operator of Oak Ridge National Laboratory under Contract No. DE-AC05-00OR22725 with the United States Department of Energy.  The United States Government has certain rights in this Data Set.  Neither UT-BATTELLE, LLC NOR THE UNITED STATES DEPARTMENT OF ENERGY, NOR ANY OF THEIR EMPLOYEES, MAKES ANY WARRANTY, EXPRESS OR IMPLIED, OR ASSUMES ANY LEGAL LIABILITY OR RESPONSIBILITY FOR THE ACCURACY, COMPLETENESS, OR USEFULNESS OF THE DATA SET


Import Python libraries and functions from other scripts

In [7]:
# Basic numeric libraries
import numpy as np
import math as m
import pandas as pd
pd.options.mode.chained_assignment = None
#pd.options.display.max_columns = 20
from scipy import stats
from __future__ import division

# Library to handle geometric objects: 
from shapely.geometry import Point, MultiPoint, box, Polygon 

# Libraries for data visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)

#Libraries for handling temporal data or monitoring processing time
import datetime as dt
import time

# Libraries for statistical analysis
import statsmodels.api as sm
import statsmodels.formula.api as smf

#System libraries
import utm
import sys
import random
from __future__ import division
import os
import csv


Key definitions:
- **urbgrid**: Grid of datacells or pixels to wich data (orders, customer locations, population, etc) will be mapped.
+ **pixel**: Each cell within the grid. Default size: 1 sq.km.


### 1. City Settings: Select city and specify grid origin point

In [31]:
#city =  'Sao Paulo'
city =  'Denver'
#city =  'San Francisco'

#Dictionaries below allow for handling several cities at the same time

Define the coordinates the lower-left corner of the city grid.

In [32]:
origins = pd.DataFrame({'lat': (-23.929093, 39.158596, 36.603832),
                        'lon': (-47.071040, -105.83495, -122.79453)},
                       index = ['Sao Paulo','Denver','San Francisco'])


# Conver Lat-lon coordinates to UTM coordinates. This is conversion is necesary to project the lat-lon system in a 
# 2 dimensional system.

for index, row in origins.iterrows():
    [east, north, zone_n, zone_l] = utm.from_latlon(row['lat'], row['lon'])
    origins.ix[index,'utm_n'] = north
    origins.ix[index, 'utm_e'] = east
    origins.ix[index, 'utm_z_n'] = zone_n
    origins.ix[index, 'utm_z_l'] = zone_l

print origins

                     lat        lon         utm_n          utm_e  utm_z_n  \
Sao Paulo     -23.929093  -47.07104  7.352078e+06  289206.026809     23.0   
Denver         39.158596 -105.83495  4.334708e+06  427861.373250     13.0   
San Francisco  36.603832 -122.79453  4.050945e+06  518376.333318     10.0   

              utm_z_l  
Sao Paulo           K  
Denver              S  
San Francisco       S  


Create a bounding box around the city. The dictionary xy_dim specifies the vertical and horizontal extension of the box. 

In [33]:
xy_dim = pd.DataFrame({'x_dim': (100, 144, 144),
                       'y_dim': (75, 176, 176)},
                      index = ['Sao Paulo','Denver','San Francisco'])

In [34]:
origin_city = Point(origins.loc[city]['utm_e'], origins.loc[city]['utm_n'])

citybox = box(origin_city.x, origin_city.y, 
              origin_city.x + xy_dim.loc[city]['x_dim']*1000, origin_city.y + xy_dim.loc[city]['y_dim']*1000)
print 'Bounding box created for', city

Bounding box created for Denver


Next, we define a fountion to generate the multiple grids needed

In [35]:
'''
To create a grid of (x_dim)*(y_dim) pixels/cells
Input
- x_dim: horizontal pixel dimension
- y_dim: vertical pixel dimension
- city_box: city's bounding box
- margin: Default = 0. useful for small areas only. It helps accommodate factions of population data in the borders of the cell.
    
Output 
- grid: 
'''

def getgrid(dim_x, dim_y, citybox, margin = 0):
    
    # Exterior coordinates of citybox are given starting at the bottom right corner 
    # of the box (x1, y0), countercklecwise: (x1, y0), (x1,y1), (x0, y1), (x0,y0) 
    #& again (x1, y0) 
    
  
    
    #Start grid from bottom-left corner 
    corner = Point(citybox.exterior.coords[3])
    origin = Point(citybox.exterior.coords[3])
    grid = [[]]
    j=0 #
    #i=0 # 
    while (corner.y+ dim_y <= citybox.bounds[3] + margin):
        while (corner.x+ dim_x<= citybox.bounds[2]+ margin):
            grid[j].append(Polygon([(corner.x, corner.y),
                                    (corner.x+dim_x, corner.y),
                                    (corner.x+dim_x, corner.y+dim_y),
                                    (corner.x, corner.y+dim_y)]))
            corner = Point(corner.x+dim_x,corner.y)
        j+=1
        origin = Point(citybox.exterior.coords[3])
        corner = Point(origin.x, origin.y + j*dim_y)
        if corner.y+dim_y <= (citybox.bounds[3]+margin):
            grid.append([])
    print 'Grid created'
    return grid

Define pixel size. For now: 1000 m x 1000 m. 

In [36]:
pix_dim_x = 1000
pix_dim_y = 1000

# Create urbgrid
urbgrid = getgrid(pix_dim_x, pix_dim_y, citybox)

Grid created


## 2. Read and process population data from LandScan
Population counts will be maped to each pixel/datacell:
1. Read population data from LandScan
+ Create **lsgrid** to standarize LandScan cell sizes (see explanation below)

Reading LandScan population for each city

In [37]:

if city == 'Sao Paulo':
    city_pop = pd.read_csv('input/saopaulo-pop.csv')
elif city == 'Denver':
    city_pop = pd.read_csv('input/denver-pop.csv')
elif city == 'San Francisco':
    city_pop = pd.read_csv('input/sfo-pop.csv')

for index, row in city_pop.iterrows():
    [east, north, zone_n, zone_l] = utm.from_latlon(row['lat'], row['lon'])
    city_pop.ix[index,'utm_n'] = north
    city_pop.ix[index,'utm_e'] = east
    city_pop.ix[index,'utm_z_n'] = zone_n
    city_pop.ix[index,'utm_z_l'] = zone_l
#print the first few row to verify
city_pop.head()


,lat,lon,population,utm_n,utm_e,utm_z_n,utm_z_l
0,40.770833,-105.412500,0,4.513400e+06,465188.193833,13.0,T
1,40.770833,-105.404167,0,4.513396e+06,465891.461591,13.0,T
2,40.770833,-105.395833,0,4.513393e+06,466594.737680,13.0,T
3,40.770833,-105.387500,0,4.513390e+06,467298.005225,13.0,T
4,40.770833,-105.379167,0,4.513387e+06,468001.272666,13.0,T


Create grid to standarize LandScan cells. The need for standardized cells arises form the fact that the exact landscan cell size varies by lattitude and longitude( even within the same city, the size of each cell varies). Then we need to define an 'averag' landscan cell size to creat our grid. 

The size of the "standard" LandScan cell size varies by city. For this first iteration, this analysis was done offline. 

Refer to LandScan's site for more information: http://web.ornl.gov/sci/landscan/landscan_faq.shtml#04

In [38]:
#The location of Sao Paulo results in lat-long cells of aporx. 850*915. For San Jose - SFO those are # 738 * 927

ls_dim = pd.DataFrame({'cel_dim_x': (850, 715, 738),
                       'cel_dim_y': (915, 925, 930)},
                      index = ['Sao Paulo','Denver','San Francisco'])

# Buffer to capture those cell that fall <500 m of the border 
# box_margin = 500

lsgrid = getgrid(ls_dim.loc[city,'cel_dim_x'], ls_dim.loc[city,'cel_dim_y'], citybox)

Grid created


Map landscan centroids to **lsgrid**. Since **lsgrid** is a set of poligons that cannot contain data, we generate a parallel matrix pop_losgrid with the mapped population values. We should not expect a 100% mapping since the area selected from Landscan can't be specified with precision and might differ in size from the area used to create the city bounding box. We expect these differences to be irrelevant.

In [40]:
#Extract Population, utem_n and utm_e to a 2-dim array
city_pop_asg = city_pop[['population','utm_e', 'utm_n']].as_matrix()

pop_lsgrid = [[]]
for j in xrange(len(lsgrid)): # to iterate over all rows of the grid
    # filter orders within row j using the utm_n colum (x[2])
    filtered_cell = filter(
        lambda x: x[2]>=lsgrid[j][0].bounds[1] and x[2]<lsgrid[j][0].bounds[3], city_pop_asg)
    #filtered points are filtered again and assigned to a cell    
    for i in xrange(len(lsgrid[0])): #to iterate over all pixels (columns) within each row
        # filter orders within row j using the utm_n colum (x[2]) 
        assigned_cell = filter(
            lambda x: x[1]>=lsgrid[0][i].bounds[0] and x[1]<lsgrid[0][i].bounds[2], filtered_cell)
        # create the orders-grid containing the number of orders in each pixel
        pop_lsgrid[j].append(sum(assigned_cell[i][0] for i in xrange (len(assigned_cell))))
    if j < len(lsgrid)-1:
        pop_lsgrid.append([])

In [41]:
print "Total population imported from LandScan:", city_pop['population'].sum()
print "Total population mapped to lsgrid:", sum(map(sum, pop_lsgrid))
print "% Population mapped", (sum(map(sum, pop_lsgrid))/city_pop['population'].sum())*100, "%"

Total population imported from LandScan: 3533965
Total population mapped to lsgrid: 3533102.0
% Population mapped 99.9755798374 %


## 3. Mapping LandScan population to pixels


Once LandScan population counts have been mapped to a standard (**lsgrid**) grid, we can now map them to the pixels (**urbgrid**). Similarly, we generate a new matrix pop_urbgrid (same size as urbgrid). 

In [42]:
'''
To map data from a landscan grid to a grid of pixels. Since the size of the landscan grid cell is equal or smaller than 1 sq.km. (pixel size),
we need to iterate over each ls cell and assign the population proportion accordingly. 
Input:
- small_pix_grid (polygons): This is generally the landscan polygons grid, aka 'lsgrid'
- big_pix_grid (polygons): This is the grid of polygons to which population data will be mapped, aka 'urbgrid'. 
- population_grid (data): population informatio to be mapped, aka pop_lsgrid

Output:
- pop_mapped_grid(data): grid of pixels containing population data. Size should be equal as big_pix_grid.  

'''


def MapPopulationtoGrid(small_pix_grid, big_pix_grid, population_grid):
    
    print 'Start time:', time.ctime()
    

    #Initialize indexes for big_pix_grid j, i
    i=0
    j=0
    # matrix to store each pixel's population count
    pop_mapped_grid = [[0 for x in xrange (len(big_pix_grid[0]))] for y in xrange (len(big_pix_grid))]
    
    
    
    #iterate over the landscan grid
    for y in xrange (len(small_pix_grid)):
        #print 'y', y
        if (i < (len(big_pix_grid)-1)):
            j=0
            for x in xrange (len(small_pix_grid[y])):
                #print 'x', x
                if (small_pix_grid[y][x].intersects(big_pix_grid[i+1][j]) == False):
                    #print 'Case l'
                    #case 1: lscell only within one urbgrid row
                    if ((small_pix_grid[y][x].within(big_pix_grid[i][j]) == True) or (j == (len(big_pix_grid[i])-1))):
                        #print 'Case 1.1'
                        #case 1.1: data cell within landscan cell"
                        #Assign the entire population of this lscell to the corresponding urbgrid cell
                        pop_mapped_grid[i][j] += population_grid[y][x]

                    elif (small_pix_grid[y][x].intersects(big_pix_grid[i][j]) == True) and (small_pix_grid[y][x].intersects(big_pix_grid[i][j+1]) == True):
                        #print 'Case 1.2'
                        #"case 1.2: lscell intersects 2 landscan cells horizontally"
                        area_inters = small_pix_grid[y][x].intersection(big_pix_grid[i][j])
                        area_inters_next = small_pix_grid[y][x].intersection(big_pix_grid[i][j+1])
                        #Assign population by the corresponding fraction
                        pop_mapped_grid[i][j] += (((area_inters.area)/(small_pix_grid[y][x].area))*(population_grid[y][x]))
                        pop_mapped_grid[i][j+1] += (((area_inters_next.area)/(small_pix_grid[y][x].area))*(population_grid[y][x]))
                        
                        #to move one column in the big_pix_grid, after reaching an intersection
                        if j <= (len(big_pix_grid[i]) - 2):
                            j+=1
                        else:
                            #Finish iteration over the row
                            x = len(small_pix_grid[y])
                else:
                    #print 'Case 2'
                    #case 2: lscell intersect two urbgird cell rows
                    if (j < (len(big_pix_grid[i])-1)):
                        if (small_pix_grid[y][x].intersects(big_pix_grid[i][j+1]) == True):
                            #case 2.2: lscell intersects upper row and 2 urbgrid cells horizontally"
                            area_inters = small_pix_grid[y][x].intersection(big_pix_grid[i][j])
                            area_inters_next = small_pix_grid[y][x].intersection(big_pix_grid[i][j+1])
                            area_inters_up = small_pix_grid[y][x].intersection(big_pix_grid[i+1][j])
                            area_inters_up_next = small_pix_grid[y][x].intersection(big_pix_grid[i+1][j+1])
                            
                            pop_mapped_grid[i][j] += (((area_inters.area)/(small_pix_grid[y][x].area))*(population_grid[y][x]))
                            
                            pop_mapped_grid[i][j+1] += ((area_inters_next.area)/(small_pix_grid[y][x].area))*(population_grid[y][x])
                            
                            pop_mapped_grid[i+1][j] += ((area_inters_up.area)/(small_pix_grid[y][x].area))*(population_grid[y][x]) 
                            
                            pop_mapped_grid[i+1][j+1] += ((area_inters_up_next.area)/(small_pix_grid[y][x].area))*(population_grid[y][x])
                            
                            
                            #to move one column in the big_pix_grid, after reaching an intersection
                            if j <= (len(big_pix_grid[i]) - 2):
                                j+=1
                            else:
                                #Finish iteration over the row
                                x = len(small_pix_grid[y])
                                
                        else:
                            #print "case 2.1: ls cell overlaps with upper row urbgrid cells but there is no horizontal intersection"
                            area_inters = small_pix_grid[y][x].intersection(big_pix_grid[i][j])
                            area_inters_up = small_pix_grid[y][x].intersection(big_pix_grid[i+1][j])
                            pop_mapped_grid[i][j] += ((area_inters.area)/(small_pix_grid[y][x].area))*(population_grid[y][x])
                            pop_mapped_grid[i+1][j] += ((area_inters_up.area)/(small_pix_grid[y][x].area))*(population_grid[y][x])
                            
                    
                    elif (j == (len(big_pix_grid[i])-1)):
                            #print "case 2.3 when reaching the horizontal border"
                            area_inters = small_pix_grid[y][x].intersection(big_pix_grid[i][j])
                            area_inters_up = small_pix_grid[y][x].intersection(big_pix_grid[i+1][j])
                            pop_mapped_grid[i][j] += ((area_inters.area)/(small_pix_grid[y][x].area))*(population_grid[y][x])
                            pop_mapped_grid[i+1][j] += ((area_inters_up.area)/(small_pix_grid[y][x].area))*(population_grid[y][x])
                            
                            
                    # To move one row up in the big_pix_grid only ofter after the loop signals an intersection across two row
                    if (x == (len(small_pix_grid[y])-1)):
                        i+=1
                        #print 'i',i
                        
                        
            pop_mapped_grid.append([])

        #For the topmost row in the big_pix_grid, there can't be intersections with another upper row, then only a subset 
        #of the cases (i.e. 1.1. and 1.2 apply)
        elif (i == (len(big_pix_grid)-1)):
            #print 'top row'
            j=0
            #"Special case: top row of big_pix_grid
            for x in xrange (len(small_pix_grid[y])):
                #case 1: datcells only within one landscan cell rows
                if ((small_pix_grid[y][x].within(big_pix_grid[i][j]) == True) or (j == (len(big_pix_grid[i])-1))):
                    pop_mapped_grid[i][j] += population_grid[y][x]

                elif (small_pix_grid[y][x].intersects(big_pix_grid[i][j]) == True) and (small_pix_grid[y][x].intersects(big_pix_grid[i][j+1]) == True):
                    #print "case 1.2X: data cell intersects 2 landscan cells horizontally"
                    area_inters = small_pix_grid[y][x].intersection(big_pix_grid[i][j])
                    area_inters_next = small_pix_grid[y][x].intersection(big_pix_grid[i][j+1])
                    pop_mapped_grid[i][j] += (((area_inters.area)/(small_pix_grid[y][x].area))*(population_grid[y][x]))
                    pop_mapped_grid[i][j+1] += (((area_inters_next.area)/(small_pix_grid[y][x].area))*(population_grid[y][x]))
    
                    if j <= (len(big_pix_grid[i]) - 2):
                        j+=1
                else:
                    pop_mapped_grid[y].append(0)

            if (y < (len(big_pix_grid))-1):
                pop_mapped_grid.append([])
                
    print 'End time:', time.ctime()
                
    return pop_mapped_grid

In [43]:
pop_urbgrid = MapPopulationtoGrid(lsgrid, urbgrid, pop_lsgrid)

Start time: Fri Dec  2 15:49:19 2016
End time: Fri Dec  2 15:50:04 2016


In [44]:
sum_pop_lsgrid = sum(map(sum, pop_lsgrid))
sum_pop_urbgrid = sum(map(sum, pop_urbgrid))

print "Total population mapped to lsgrid:", sum_pop_lsgrid
print "Total population mapped to urbgrid:", sum_pop_urbgrid
print "% Population mapped", sum_pop_urbgrid/ sum_pop_lsgrid*100, "%"

Total population mapped to lsgrid: 3533102.0
Total population mapped to urbgrid: 3533102.0
% Population mapped 100.0 %


## 4. Process and export results

Generate a dataframe of pixels

In [45]:
# to convert both grids to a list of sq.kms. (ie. pixels) with lat-lon data 
def get_pixel_list(urbgrid, pop_urbgrid):
    print 'Start time:', time.ctime()
    pixel_list = [[0 for j in range(0,6)] for k in xrange(len(urbgrid)*len(urbgrid[0]))]
    k = 0 
    for i in xrange(len(urbgrid)):
        for j in xrange(len(urbgrid[0])): 
            (Lat, Lon) = utm.to_latlon(urbgrid[i][j].centroid.x, urbgrid[i][j].centroid.y,
                                     origins.loc[city]['utm_z_n'], origins.loc[city]['utm_z_l'] )
            pixel_list[k][1] = Lat                         #Latiude in decimal degrees   
            pixel_list[k][2] = Lon                         #Longitude in decimal degrees
            pixel_list[k][3] = urbgrid[i][j].centroid.y    #Northing   
            pixel_list[k][4] = urbgrid[i][j].centroid.x    #Easting
            pixel_list[k][5] = round(pop_urbgrid[i][j])    # population
            k+=1
    pixel_list_df = pd.DataFrame(pixel_list)
    pixel_list_df.columns = ['pixel_ID','lat', 'lon', 'utm_n', 'utm_e','population']
    pixel_list_df['pixel_ID'] = pixel_list_df.index.values
    
    pixel_list_df[['pixel_ID','population']] = pixel_list_df[['pixel_ID','population']].astype(int)
    print 'End time:', time.ctime()
    return pixel_list_df

###
# The polygon of each pixel could also be printed by adding:
#datacell_list[k][5] = urbgrid[i][j]         

In [46]:
pixel_list = get_pixel_list(urbgrid, pop_urbgrid)

Start time: Fri Dec  2 15:50:04 2016
End time: Fri Dec  2 15:50:16 2016


In [47]:
#Export results 
if city == 'Sao Paulo':
    pixel_list.to_csv('output/SP_pixels_population.csv', index = False)
elif city == 'Denver':
    pixel_list.to_csv('output/Denver_pixels_population.csv', index = False)
elif city == 'San Francisco':
    pixel_list.to_csv('output/SFO_pixels_population.csv', index = False)